<a href="https://colab.research.google.com/github/geun-hyoung/Forecasting_the_number_of_complaints_received/blob/main/Data_Clenaing%26EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Data Load(Concat)& Library 

In [1]:
# google drive
from google.colab import drive
drive.mount('/content/drive') # Connect to drive

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
factor_1 = pd.read_csv('/content/drive/MyDrive/데이터마이닝/112factor_19.csv', encoding = 'cp949')
factor_2 = pd.read_csv('/content/drive/MyDrive/데이터마이닝/112factor_20.csv', encoding = 'cp949')
factor_3 = pd.read_csv('/content/drive/MyDrive/데이터마이닝/112factor_21.csv', encoding = 'cp949')
factor_df = pd.concat([factor_1, factor_2, factor_3], axis = 0)
factor_df.head()

,ACPT_YM,PLCST_NM,RPRT_CASCNT_WHOL,RPRT_SEX_MALE,RPRT_SEX_FMLE,RPRT_SEX_UNKWN,RPRT_SEX_ETC,ACPT_EMERG_TYPE_EMERG,ACPT_EMERG_TYPE_IMPRTNC,ACPT_EMERG_TYPE_GNRL,...,CFINC_ETC,OCRN_DOTW_SNDD,OCRN_DOTW_MON,OCRN_DOTW_TUE,OCRN_DOTW_WED,OCRN_DOTW_THU,OCRN_DOTW_FRI,OCRN_DOTW_SAT,MNTH_AVE_INCOME_AMT,TOTL_RESID_PUL_CNT
0,201901,원주,5014,2916,1500,598,0,14,1090,2899,...,4491,611,617,778,802,813,702,691,NaN,NaN
1,201901,음성,1650,904,463,283,0,10,438,823,...,1509,183,213,250,274,254,227,249,NaN,NaN
2,201901,의령,278,156,73,49,0,1,44,146,...,265,28,29,53,47,42,40,39,NaN,NaN
3,201901,의성,547,287,145,115,0,2,152,247,...,509,67,65,71,88,108,67,81,NaN,NaN
4,201901,의왕,1735,928,501,305,1,23,453,823,...,1637,205,229,262,288,322,218,211,NaN,NaN


In [4]:
factor_df.shape

(9934, 28)

In [5]:
reported_19 = pd.read_csv('/content/drive/MyDrive/데이터마이닝/Korea_112_19.csv', encoding = 'cp949')
reported_20 = pd.read_csv('/content/drive/MyDrive/데이터마이닝/Korea_112_20.csv', encoding = 'cp949')
reported_21 = pd.read_csv('/content/drive/MyDrive/데이터마이닝/Korea_112_21.csv', encoding = 'cp949')
reported_df = pd.concat([reported_19, reported_20, reported_21], axis = 0)
reported_df.tail()

,ACPT_YM,PLCST_NM,RPRT_CASCNT_WHOL,RPRT_SEX_MALE,RPRT_SEX_FMLE,RPRT_SEX_UNKWN,RPRT_SEX_ETC,ACPT_EMERG_TYPE_EMERG,ACPT_EMERG_TYPE_IMPRTNC,ACPT_EMERG_TYPE_GNRL,...,OCRN_TIZN_14,OCRN_TIZN_15,OCRN_TIZN_16,OCRN_TIZN_17,OCRN_TIZN_18,OCRN_TIZN_19,OCRN_TIZN_20,OCRN_TIZN_21,OCRN_TIZN_22,OCRN_TIZN_23
3286,202112,영천,994,484,263,247,0,3,170,513,...,78,65,52,57,51,62,66,60,46,58
3287,202112,대구청,128,68,34,26,0,4,55,51,...,5,2,9,2,12,9,9,7,5,5
3288,202112,평창,544,257,118,169,0,4,128,278,...,38,23,48,28,33,30,23,26,33,23
3289,202112,대전청,111,55,31,25,0,1,71,28,...,3,6,4,5,8,9,6,6,10,5
3290,202112,제주청,39,17,12,10,0,0,26,11,...,2,3,1,1,5,0,1,2,5,0


In [6]:
reported_df.shape

(9946, 43)

In [7]:
incheon_factor = factor_df[factor_df['PLCST_NM'].str.contains('인천')] 
incheon_df = reported_df[reported_df['PLCST_NM'].str.contains('인천')] 

In [8]:
def change_dt(values):
    return str(values)[0:4] + '-' + str(values)[4:] 

incheon_df['ACPT_YM'] = incheon_df['ACPT_YM'].apply(change_dt)
incheon_factor['ACPT_YM'] = incheon_factor['ACPT_YM'].apply(change_dt)

<ipython-input-8-7f319e672a1b>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  incheon_df['ACPT_YM'] = incheon_df['ACPT_YM'].apply(change_dt)
<ipython-input-8-7f319e672a1b>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  incheon_factor['ACPT_YM'] = incheon_factor['ACPT_YM'].apply(change_dt)


In [9]:
import datetime as dt
incheon_df['ACPT_YM'] = incheon_df['ACPT_YM'].apply(lambda x: dt.datetime.strptime(x, "%Y-%m"))
incheon_factor['ACPT_YM'] = incheon_factor['ACPT_YM'].apply(lambda x: dt.datetime.strptime(x, "%Y-%m"))

<ipython-input-9-3dc3e4d06191>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  incheon_df['ACPT_YM'] = incheon_df['ACPT_YM'].apply(lambda x: dt.datetime.strptime(x, "%Y-%m"))
<ipython-input-9-3dc3e4d06191>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  incheon_factor['ACPT_YM'] = incheon_factor['ACPT_YM'].apply(lambda x: dt.datetime.strptime(x, "%Y-%m"))


In [10]:
incheon_factor.dtypes

ACPT_YM                    datetime64[ns]
PLCST_NM                           object
RPRT_CASCNT_WHOL                    int64
RPRT_SEX_MALE                       int64
RPRT_SEX_FMLE                       int64
RPRT_SEX_UNKWN                      int64
RPRT_SEX_ETC                        int64
ACPT_EMERG_TYPE_EMERG               int64
ACPT_EMERG_TYPE_IMPRTNC             int64
ACPT_EMERG_TYPE_GNRL                int64
ACPT_EMERG_TYPE_CVAPP               int64
ACPT_EMERG_TYPE_ETC                 int64
CFINC_DSTVC                         int64
CFINC_DTVC                          int64
CFINC_MRDR                          int64
CFINC_SVLNC                         int64
CFINC_SUCD                          int64
CFINC_INCVLC                        int64
CFINC_ETC                           int64
OCRN_DOTW_SNDD                      int64
OCRN_DOTW_MON                       int64
OCRN_DOTW_TUE                       int64
OCRN_DOTW_WED                       int64
OCRN_DOTW_THU                     

In [16]:
incheon_df.to_csv('/content/drive/MyDrive/데이터마이닝/인천시신고데이터.csv', sep=',', na_rep='NaN')   # missing data representation (결측값 표기)
incheon_factor.to_csv('/content/drive/MyDrive/데이터마이닝/인천시융합데이터.csv', sep=',', na_rep='NaN')

In [22]:
incheon_factor.PLCST_NM.value_counts()

인천강화         36
인천계양         36
인천남동         36
인천논현         36
인천미추홀        36
인천부평         36
인천삼산         36
인천서부         36
인천연수         36
인천중부         36
인천청          36
인천국제공항경찰단    24
Name: PLCST_NM, dtype: int64

In [28]:
ic = incheon_df.groupby(incheon_df['PLCST_NM']).count()

In [30]:
print(ic)

           ACPT_YM  RPRT_CASCNT_WHOL  RPRT_SEX_MALE  RPRT_SEX_FMLE  \
PLCST_NM                                                             
인천강화            36                36             36             36   
인천계양            36                36             36             36   
인천국제공항경찰단       24                24             24             24   
인천남동            36                36             36             36   
인천논현            36                36             36             36   
인천미추홀           36                36             36             36   
인천부평            36                36             36             36   
인천삼산            36                36             36             36   
인천서부            36                36             36             36   
인천연수            36                36             36             36   
인천중부            36                36             36             36   
인천청             36                36             36             36   

           RPRT_SEX

In [ ]:
# 소득순위랑 이런거 drop하기
# 전체연도별 사건 접수량
# 요일별 신고 접수량
# 연도별, 시간대별 신고 접수량
#연도별, 경차서별 신고 접수량